In [1]:
import pandas as pd
import numpy as np
import os

# Cases

In [2]:
os.getcwd()

'C:\\Users\\Michael\\jupyter_books\\projects\\covid\\notebooks\\CDCnotebooks'

In [3]:
path_to_test_results = '../../data/CDCdata/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv'
chunksize = 250000
non_continental_states = ['AK', 'HI', 'MP', 'GU', 'VI', 'PR', 'AS']


raw_iter = pd.read_csv(path_to_test_results, chunksize=chunksize)

In [4]:
def process_chunk(chunk):
    chunk = chunk.replace('Missing', np.nan)
    chunk['case_month'] = pd.to_datetime(chunk['case_month'], format='%Y-%m').dt.to_period('M')
    tests_noCounty = chunk[chunk['county_fips_code'].isna()]
    chunk = chunk.drop(tests_noCounty.index)
    non_continentals = chunk[chunk['res_state'].isin(non_continental_states)]
    chunk = chunk.drop(non_continentals.index)
    chunk['county_fips_code'] = chunk['county_fips_code'].astype(int).astype(str).str.zfill(5)
    
    cases = chunk.groupby(['county_fips_code', 'case_month'])['current_status'].count()
    cases.index = cases.index.rename(['FIPS', 'month'])
    cases.name = 'case_count'
    
    dropped_cases = tests_noCounty.groupby(['res_state', 'case_month'])['current_status'].count()
    dropped_cases.index = dropped_cases.index.rename(['state', 'month'])
    dropped_cases.name = 'case_count'

    return cases, dropped_cases

In [5]:
cases = pd.DataFrame({'case_count': []}, index=pd.MultiIndex.from_arrays([[],[]], names=('FIPS', 'month')))
dropped_cases = pd.DataFrame({'case_count': []}, index=pd.MultiIndex.from_arrays([[],[]], names=('state', 'month')))

for chunk in raw_iter:
    new_cass, new_drpd_cass = process_chunk(chunk)
    
    cases = pd.merge(cases, new_cass, how='outer', on=['FIPS', 'month']).fillna(0.0, downcast='infer')
    cases['case_count'] = cases['case_count_x'] + cases['case_count_y']
    cases = cases.drop(['case_count_x', 'case_count_y'], axis=1)
    
    dropped_cases = pd.merge(dropped_cases, new_drpd_cass, how='outer', on=['state', 'month']).fillna(0.0, downcast='infer')
    dropped_cases['case_count'] = dropped_cases['case_count_x'] + dropped_cases['case_count_y']
    dropped_cases = dropped_cases.drop(['case_count_x', 'case_count_y'], axis=1)

C:\Users\Michael\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
cases = cases.sort_index()
cases

case_count
FIPS  month              
01001 2020-03          16
      2020-04          40
      2020-05         154
      2020-06         326
      2020-07         480
...                   ...
55141 2021-03          97
      2021-04         229
      2021-05         180
      2021-06          52
      2021-07          23

[23912 rows x 1 columns]

In [7]:
cases_unstacked = cases.unstack()
cases_unstacked

case_count                                                          \
month    2020-01 2020-02 2020-03 2020-04 2020-05 2020-06 2020-07 2020-08   
FIPS                                                                       
01001        NaN     NaN    16.0    40.0   154.0   326.0   480.0   387.0   
01003        NaN     NaN    62.0   135.0    80.0   436.0  2131.0  1369.0   
01005        NaN     NaN     NaN    41.0    98.0   149.0   232.0   140.0   
01007        NaN     NaN     NaN    35.0    29.0    99.0   193.0   175.0   
01009        NaN     NaN    13.0    32.0    15.0   144.0   559.0   539.0   
...          ...     ...     ...     ...     ...     ...     ...     ...   
55133        NaN     NaN   192.0   212.0   355.0   554.0  2176.0  1682.0   
55135        NaN     NaN     NaN    11.0    29.0    76.0   241.0   303.0   
55137        NaN     NaN     NaN     NaN     NaN    20.0    60.0    61.0   
55139        NaN     NaN    24.0    73.0   243.0   356.0   366.0   513.0   
55141        NaN     NaN     NaN     NaN     NaN    58.0   161.0   232.0   

                                                                        \
month 2020-09 2020-10  2020-11 2020-12 2021-01 2021-02 2021-03 2021-04   
FIPS                                                                     
01001   331.0   393.0    733.0  1412.0  1499.0   562.0   235.0   150.0   
01003  1340.0   936.0   2056.0  4060.0  4499.0  1526.0   509.0   509.0   
01005   147.0   155.0    118.0   274.0   435.0   164.0    78.0    70.0   
01007   114.0   204.0    340.0   548.0   470.0   120.0    67.0    47.0   
01009   298.0   517.0   1087.0  1597.0  1002.0   327.0   288.0   143.0   
...       ...     ...      ...     ...     ...     ...     ...     ...   
55133  2279.0  5494.0  12150.0  8443.0  4237.0  1349.0  1849.0  1825.0   
55135   457.0  1215.0   1080.0   454.0   463.0   184.0   180.0   166.0   
55137   266.0   524.0    364.0   171.0   136.0    55.0    39.0    51.0   
55139  1827.0  3968.0   3690.0  2207.0  1443.0   559.0  1320.0  1151.0   
55141   408.0  1184.0   2067.0  1246.0   695.0   226.0    97.0   229.0   

                               
month 2021-05 2021-06 2021-07  
FIPS                           
01001   223.0    59.0   107.0  
01003   619.0   382.0   770.0  
01005    39.0    11.0    40.0  
01007    73.0    37.0    57.0  
01009   345.0    56.0   111.0  
...       ...     ...     ...  
55133   569.0   218.0   261.0  
55135    74.0    55.0    60.0  
55137    74.0    24.0     NaN  
55139   607.0   329.0   164.0  
55141   180.0    52.0    23.0  

[1522 rows x 19 columns]

In [8]:
if os.path.exists('../../pickles/CDCpickles/cases.pkl'):
    os.remove('../../pickles/CDCpickles/cases.pkl')
cases.to_pickle('../../pickles/CDCpickles/cases.pkl')